# Multimessenger inference

> Note: this tutorial has the same dependencies as tutorial `2-parameter_estimation`, so you can run it in the same environment.

In this tutorial you will:
* Understand how arbitrary priors can be defined, instantiated and combined in `cogwheel`.
* Infer the parameters of the binary neutron star GW170817, knowing its counterpart's sky location and redshift.

This tutorial is intentionally less detailed that the previous ones, to encourage you to familiarize with browsing a code's documentation and source.

## Motivation

Observational astronomers were able to identify the kilonova explosion that resulted from the binary neutron star merger GW170817.
This allowed to accurately pinpoint the sky location of the source (as opposed to inferring it from the gravitational wave data).
Moreover, the redshift of the host galaxy has also been determined observationally. We can use this to fix the luminosity distance of the source (if we assume the expansion rate of the universe $H_0$ is known, e.g. from the Planck measurements of the cosmic microwave background). Alternatively, we could measure the luminosity distance from the gravitational-wave data and infer $H_0$.

In addition, there are good physical arguments that constrain the spin of neutron stars. An absolute limit $\chi \lesssim 0.7$ can be placed, since beyond that the centrifugal force is so large that a neutron star would break up, shedding mass equatorially.
Neutron stars are spun up by accretion from a companion star, and spin down in the absence of accretion. Merging neutron stars have likely been for a long time in orbit with a compact companion, and spun down.
Including this information in the inference as a spin prior could help constrain even other parameters (e.g. mass ratio), which are correlated.
(This was implemented as the "low-spin" prior in the GW170817 [paper](https://arxiv.org/pdf/1710.05832).)

## Your mission

1. Define a prior for the sky location that sets the right ascension and declination to a fixed value passed by the user.
1. Likewise for the luminosity distance.
1. Define a spin prior that allows you to set the maximum dimensionless spin of the component objects.
1. Define a combined prior over the full set of parameters: masses, spins, sky localization, arrival time, polarization, inclination, orbital phase, distance, tidal deformabilities.
1. Download GW data for GW170817.
1. Instantiate a posterior object using your custom prior, with the right ascension, declination and distance corresponding to GW170817's host galaxy.
1. Instantiate a sampler and estimate the parameters of GW170817.

## Short background

In a `cogwheel` inference run, the prior distribution is an instance of a `Prior` class. The prior class defines a family of prior distributions (say, "normal distribution") and the class instance a completely specified distribution ("normal distribution with mean 1 and variance 2").

`cogwheel` doesn't offer a pre-specified way of expressing that the sky location is known. You will need to do that by constructing a "delta-function" prior on the right ascension `ra` and declination `dec`. Note that in that case `ra` and `dec` become standard parameters that do not have any sampled parameters associated.

In the case we are studying, we want to fix the sky location and distance so we cannot employ extrinsic-parameter marginalization as in the previous tutorials. We will have to sample all the non-fixed parameters. The total set of standard parameters we ultimately need to produce are:

In [ ]:
import cogwheel.waveform

cogwheel.waveform.WaveformGenerator.params

`cogwheel` allows to define modular priors (for few variables each) and combine them, so you will only need to define new classes for the specific ones you want to modify.

* Go through the `make_your_own_prior.ipynb` tutorial in the `cogwheel` repository.

* You are now in shape to start defining your own priors.

## 1. Define a new `FixedSkyLocationPrior` prior class.

Operationally, this is very similar to what `cogwheel` already does for the reference frequency: it is set to a fixed value that is not known in advance but needs to be chosen by the user. Take inspiration from the `cogwheel.gw_prior.miscellaneous.FixedReferenceFrequencyPrior` implementation, whose source code you can also find in the repository.

In [ ]:
from cogwheel.prior import FixedPrior


class FixedSkyLocationPrior(FixedPrior):
    """
    Fix the right ascension and declination to user-specified values.
    """
    # Complete this

Now test it. Is this what you expect?

In [ ]:
test_skyloc_prior = FixedSkyLocationPrior(ra=0.0, dec=1.0)
print(f'{test_skyloc_prior.sampled_params = }')
print(f'{test_skyloc_prior.standard_par_dic = }')
print(f'{test_skyloc_prior.lnprior() = }')

## 2. Distance prior
* Now do a prior class that fixes the luminosity distance

## 3. Spin prior

Let's now define a spin prior that allows the user to pass a maximum spin.

### Suggestions (but you may do otherwise):

- For simplicity, restrict to aligned-spin configurations and create a prior for `s1z`, `s2z`.

  `cogwheel` already offers `cogwheel.gw_prior.spin.ZeroInplaneSpinsPrior` to set in-plane spin components to zero, so you only need to define a new class for the $z$ components in this case.

- A simple choice is to sample directly in the `s1z`, `s2z` parameters (same sampled and standard parameters). This makes the prior easy to define, but at the price of making sampled variables correlated (e.g., typically the combination $\chi_\textrm{eff}$ (effective spin) is better measured than `s1z` or `s2z`).
If you go this route, you may find the `cogwheel.prior.IdentityTransformMixin` mixin-class useful.

> Mixin classes are designed to behave well with multiple inheritance, so they are easy to combine.
> They are defined as classes that do not provide their own attributes or `__init__` method.
> They allow to pick and choose behaviors for your classes to have, in a resusable way.

* For simplicity, you may choose a uniform prior on the spins. Then, the `cogwheel.prior.UniformPriorMixin` may be useful.

## 4. Combine modular priors
We now define another prior class that corresponds to the prior over the full parameter space:

In [ ]:
from cogwheel.gw_prior.combined import (
    RegisteredPriorMixin,
    CombinedPrior,
    UniformDetectorFrameMassesPrior,
    IsotropicInclinationPrior,
    UniformTimePrior,
    UniformPolarizationPrior,
    UniformPhasePrior,
    ZeroInplaneSpinsPrior,
    UniformTidalDeformabilitiesBNSPrior,
    FixedReferenceFrequencyPrior
)


class MultiMessengerBNSPrior(RegisteredPriorMixin, CombinedPrior):
    """
    Has customized fixed-sky-location, fixed-distance, low-spin priors.
    """
    default_likelihood_class = RelativeBinningLikelihood

    prior_classes = [
        UniformDetectorFrameMassesPrior,
        IsotropicInclinationPrior,
        FixedSkyLocationPrior,
        UniformTimePrior,
        UniformPolarizationPrior,
        UniformPhasePrior,
        YourFixedDistancePriorHere,
        YourSpinPriorHere,
        ZeroInplaneSpinsPrior,
        UniformTidalDeformabilitiesBNSPrior,
        FixedReferenceFrequencyPrior,
    ]

In [ ]:
# Test it: is this what you expect?
print(f'{MultiMessengerBNSPrior.sampled_params = }')
print(f'{MultiMessengerBNSPrior.standard_params = }')

## 5. Download data
* Download data for GW170817, making sure it is the version with the LIGO-Livingston glitch cleaned.
* Create the corresponding `cogwheel.data.EventData` instance, name it `event_data`.

> See the `event_data.ipynb`tutorial in the `cogwheel` repository for inspiration.

## 6. Instantiate a `Posterior`
* Instantiate a posterior object using your custom prior, with the right ascension, declination and distance corresponding to GW170817's host galaxy NGC 4993 (assuming your favorite value of $H_0$).

As you already know from the previous tutorial, this is done with `cogwheel.posterior.Posterior.from_event()`.
However, your new priors require special initialization parameters that `cogwheel` doesn't know how to handle automatically:

Look at

In [ ]:
MultiMessengerBNSPrior.init_parameters()

Any parameters there that were introduce by you, you will need to pass to `from_event` as `prior_kwargs`, e.g.:

In [ ]:
import cogwheel.posterior

posterior = cogwheel.posterior.Posterior.from_event(
    event_data,
    mchirp_guess=1.198,
    approximant='IMRPhenomD_NRTidalv2',
    prior_class=MultiMessengerBNSPrior,
    prior_kwargs={
        'ra': NGC_4993_ra,  # Provide this value (in radians!)
        'dec': NGC_4993_dec,  # Provide this value (in radians!)
        'd_luminosity': NGC_4993_dl,  # Provide this value
        'max_spin': 0.05  # Provide this value
    }
)

Check that the reference solution looks good:

In [ ]:
posterior.likelihood.plot_whitened_wf(posterior.likelihood.par_dic_0)

## 7. Infer the parameters of GW170817
* Instantiate and run a sampler for the above `posterior`.
* Can you reproduce the inference results of the GW170817 [paper](https://arxiv.org/pdf/1710.05832)?